# <center>INVOICING SYSTEM: Pt. 1</center>

# <center>PRE INVOICE</center>

# <center>Workflow</center>

### 1. User will add data to an Excel file.
### 2. Select Invoice Number
### 3. Select Customer
### 4. Type pallet numbers that will go on the invoice
### 5. Pre Invoice work
    5.1 Check for missing values
    5.2 Add Manufacturers
    5.3 Add HS Codes
    5.4 Do the invoice
### 6. Save Pre Invoice into an Excel file

---

In [2]:
import pandas as pd
import re
import datetime
from openpyxl import load_workbook
from openpyxl.styles import PatternFill, Border, Side, Alignment, Protection, Font
from copy import copy

# Functions to use

In [3]:
'''
CHECK IF IT'S STRING FUNCTION

This function checks if the value of input are letters or numbers.
If it's letters and numbers together it will count as a word and not a number.

Purpose: Avoid crashing program by user's input
'''
def check_str(val):
    while True:
        try:
            #Converts to float because if it can be converted to float, it can also be converted to integer
            check1=float(val)
            val=input("Please don't type a number - Try again: ")
        except:
            return val

In [4]:
'''
CHECK IF IT'S NUMBER FUNCTION

This function checks if the value of input are numbers.
If it's letters and numbers together it will count as a word and not a number.

Purpose: Avoid crashing program by user's input
'''
def check_num(val):
    while True:
        try:
            if val.lower()=='done':
                return val.lower()
            else:
                #This works because you can't convert a str to an int
                check1=int(val)
                return val
        except:
            val=input("\nPlease type a number - Try again: ")
            pass

In [5]:
'''
Cell formatting - Taking format from previous cell and applying it to the cell below
'''
def format_cell(worksheet, cell):
    prev_cell=cell.column_letter+str(cell.row-1)
    cell_loc=worksheet[prev_cell]
    cell.border=copy(cell_loc.border)
    cell.alignment=copy(cell_loc.alignment)    
    cell.number_format=copy(cell_loc.number_format)  

In [6]:
'''
TRANSFER VALUES TO WORKSHEET FUNCTION

Function that will transfer all the information from the selected dataframe to the desired worksheet.
And apply the format from the cell above.
worksheet: Worksheet where information will be transfered
df: dataframe to transfer to worksheet
loc_x: Location on the X axis (letters)
loc_y: Location on the Y axis (numbers)
'''

def pass_vals(worksheet, df, loc_x=1, loc_y=1):
    for i in range(0,df.shape[0]):
        loc_y=1
        for j in range(0,df.shape[1]):
            format_cell(worksheet, \
                        worksheet[worksheet.cell(row=loc_x, column=loc_y, value=df.iloc[i,j]).coordinate])
            loc_y+=1
        loc_x+=1

In [7]:
'''
CHECK FOR MISSING VALUES FUNCTION

Checks for missing values - It helps user know if he needs to go back and modify information
'''
def check_miss(df):    
    var=0
    for cell in df:
        if cell==None:
            var+=1
        else:
            pass
    return var

### Load Pre Invoice template 

In [52]:
'''
Asks where is the container being shipped and selects correct template
Opens necessary files to get information from.
    wb: loads invoice template workbook
'''
wb = load_workbook('Pre_Invoice_Template.xlsx')

### Select Pre Invoice Number

In [53]:
"""
Saving wb['worksheet_name'] to a variable that makes that worksheet active.
ws_invoice: invoice worksheet
"""
#add worksheets to variables for future use
ws_invoice=wb['invoice']

'''
User inputs invoice number and it is written on invoice template
invoice_num: variable for invoice number
t_date: today's date
'''
#Adding today's date to invoice and packing list
d=datetime.datetime.today()
t_date=str(d.month)+'/'+str(d.day)+'/'+str(d.year)
ws_invoice.cell(row=2, column=9, value=t_date)

invoice_num=check_num(input('Type Invoice Num: '))
#invoice number is assigned to cell in invoice template and packing list
ws_invoice.cell(row=1, column=9, value=invoice_num)
print("Pre Invoice No. will be:", invoice_num)

Type Invoice Num: 4
Pre Invoice No. will be: 4


### Select Customer

In [54]:
'''
Select and add customer to invoice from customers database
    1. Computer asks for the customer's name
    2. It has to look on a different file if customer exists in the database
        2.1. If customer exists, add it's information to the invoice
            2.1.1 Create variables that will go in each part of the invoice
            2.1.2 Assign values to each variable
            2.1.3 Add variables to invoice
        2.2. If it doesn't exist, try again. Type 'exit' to exit prompt
    
cust_database: loads customers database workbook
'''

cust_database = load_workbook('Customer_database.xlsx')
ws_cust=cust_database.active

while True:
    cust_name=input("Type the customer's code: ")
    for cust_search in ws_cust.iter_rows(min_row=1, max_col=8, values_only=True): #iterates over column on different file
        cust_code=re.search("^({})$".format(cust_name.lower()), str(cust_search[0]).lower()) !=None #regex expression to look in other file for customer code
        if cust_code==False:
            pass
        else:
            print('Customer',cust_name,'found...')
            # Assign values to each variable
            name=cust_search[1]
            address=cust_search[2]
            city=cust_search[3]
            phone=cust_search[4]
            fax=cust_search[5]
            email=cust_search[6]
            terms=cust_search[7]
            print('...Transferring information...')

            #Insert variables in invoice
            ws_invoice.cell(row=2, column=2, value=name)
            ws_invoice.cell(row=3, column=2, value=address)
            ws_invoice.cell(row=4, column=2, value=city)
            ws_invoice.cell(row=5, column=2, value=phone)
            ws_invoice.cell(row=6, column=2, value=fax)
            ws_invoice.cell(row=7, column=2, value=email)
            ws_invoice.cell(row=9, column=2, value=terms)

            print('...Information transferred')
            cust_name='exit'

    if cust_name=='exit':
        break
    else:
        print('Customer code not found - Try again\n Type "exit" to close')

Type the customer's code: spare
Customer spare found...
...Transferring information...
...Information transferred


# Prepare Pre Invoice 

### Get data

In [55]:
'''
wb_rec: Workbook with all the information to transfer to invoice
info: Empty dataframe where all the information is
shipment: Empty dataframe that has all the skids that will be shipped
skid_list: Dictionary that will be populated with used skids, weight and dimension for each skid.
    It's purpose is not to have duplicated skids by comparing what's inside the list with the 
    inputed numbers, and store information to make the packing list later on.
'''
wb_rec = load_workbook('rec_2.xlsx')
wb_rec=wb_rec.active
shipment=pd.DataFrame()
skid_list=set([])
skid_dict={}

In [56]:
#Create list of unique skid numbers available in shipment
for row in wb_rec.iter_rows(min_row=1, max_col=17, values_only=True):
    skid_list.add(row[2])

In [57]:
'''
User get's asked for skid number and it is checked if it exists or if it has already been selected
and it raises an error if it does. Otherwise, it adds the information to the working dataframe
'''

while True:
    print("\nType 'done' when finished")
    skid_num=check_num(input("\nType the skid number: "))
    if skid_num!='done' and skid_num not in skid_dict and int(skid_num) in skid_list:
        for row in wb_rec.iter_rows(min_row=1, max_col=17, values_only=True):
            if int(skid_num)==row[2]:
                shipment=shipment.append(pd.Series(list(row)), ignore_index=True)
                skid_dict[skid_num]={}
            else:
                pass
    elif skid_num=='done':
        print('Finished')
        break
    elif skid_num in skid_dict:
        print('\n***You already picked this skid***')
    elif int(skid_num) not in skid_list:
        print('\n***Skid number',skid_num,"doesn't exist***")
        print('***Try again***')
    else:
        pass


Type 'done' when finished

Type the skid number: 1

Type 'done' when finished

Type the skid number: 2

Type 'done' when finished

Type the skid number: 3

Type 'done' when finished

Type the skid number: 4

Type 'done' when finished

Type the skid number: 5

Type 'done' when finished

Type the skid number: 6

Type 'done' when finished

Type the skid number: 7

Type 'done' when finished

Type the skid number: 8

Type 'done' when finished

Type the skid number: done
Finished


### Check if there are missing values in user's inputed information

In [58]:
print('***There are {} HS Codes missing*** - PLEASE FIX THIS ISSUE'.format(check_miss(shipment[13])))
print('***There are {} Country of Origin codes missing*** - PLEASE FIX THIS ISSUE'.format(check_miss(shipment[14])))

***There are 0 HS Codes missing*** - PLEASE FIX THIS ISSUE
***There are 0 Country of Origin codes missing*** - PLEASE FIX THIS ISSUE


### Add Manufacturer

In [59]:
'''
Search through dataframe and compare if the order starts with NB, if it does go to the next line.
If it's different look through each part number to determine the manufacturer. Pick the manufacturer depending
on the content of the part number.

It also tells the user if there are empty manufacturer cells.
'''
jerani=['A','B','C','D']
#Add manufacturer to dataframe (column 12)
for i,row in enumerate(shipment[0]):
    #shipment[0] has a type of object, need to convert everything to string because of nature of data
    if ((re.search('^N{}'.format(jerani), str(row))!=None)==True):
        pass
    else:
        #shipment[5] has a type of object, need to convert everything to string because of nature of data
        if (re.search('[a-zA-Z]|-', str(shipment.iloc[i,5]))):
            shipment.iloc[i,12]='AcDelco'
        else:
            shipment.iloc[i,12]='GM'

#Checks for empty cells to tell the user there is missing information
if check_miss(shipment[12])!=0:
    print('There are {} empty Manufacturer cells - PLEASE FIX THIS ISSUE'.format(check_miss(shipment[12])))
else:
    pass


### Add HS Codes

In [60]:
#Add HS Codes to dataframe (column 13)

### Worksheet Data

In [61]:
'''
Change type of columns - to ensure everything has the same type and the program doesn't 
confuse labels because of their types.
'''
shipment[1]=shipment[1].astype(str)
shipment[2]=shipment[2].astype(int)
shipment[3]=shipment[3].astype(int)
shipment[4]=shipment[4].astype(int)
shipment[5]=shipment[5].astype(str)
shipment[7]=shipment[7].astype(str)
shipment[8]=shipment[8].astype(str)
shipment[13]=shipment[13].astype(str)

### Invoice Section

In [76]:
#Add total price for each part number
total_vals=shipment[15]*shipment[4]
shipment[16]=total_vals

'''
Combine total quantity for each part number and order to get a summarised table based on order number,
part number and price.
'''

total_quantity=shipment.groupby([8,5,15], as_index=False)[4].sum()


'''
Shipment dataframe is grouped by order number and part number to combine duplicate values, the index is 
returned to get the summarized information from that particular order and part number.
The index will be passed to the original shipment dataframe and it will return the rows with the information
pointing to the unique rows.
'''
invoice_elem=shipment.loc[shipment.groupby([8,5,15], as_index=False)[4].idxmax()]
#Assign new elements to quantity, if not assigned, dataframe was taking original values and not the updated ones
invoice_elem[4]=list(total_quantity[4])


'''
final_invoice: Merge total_quantity dataframe to invoice_elem to get a dataframe with 
    the total quantities and all the information

invoice_pt1: Is a dataframe that will contain the processed information - merge invoice_elem and total_quantity
packing_list: Is a dataframe that has the summarised information regarding the packing list
'''
invoice_pt1=invoice_elem.merge(total_quantity)

# #Convert total quantities and line number to integer type
invoice_pt1[4]=invoice_pt1[4].astype(int)
invoice_pt1[3]=invoice_pt1[3].astype(int)
invoice_pt1[13]=invoice_pt1[13].astype(str)

# #Select columns that will go to the final invoice 
final_invoice=invoice_pt1.loc[:,[3,4,5,6,8,12,13,14,15,16]]
t_val=final_invoice[15]*final_invoice[4]
final_invoice[16]=t_val
    
#Paste Final Values on Invoice
t_m1=ws_invoice.cell(row=final_invoice.shape[0]+20,column=9,value='Total merchandise')
t_m1.alignment=Alignment(horizontal='right')

t_m1_val=ws_invoice.cell(row=final_invoice.shape[0]+20,column=10,value=sum(final_invoice[16]))
t_m1_val.font=Font(bold=True)
t_m1_val.number_format=copy(ws_invoice['J19'].number_format)

t_m2=ws_invoice.cell(row=final_invoice.shape[0]+21,column=9,\
                value='Forwarding, packing, pier delivery, documentation & Insurance')
t_m2.alignment=Alignment(horizontal='right')


t_m3=ws_invoice.cell(row=final_invoice.shape[0]+22,column=9,value='Final Total FOB Port of New York USA')
t_m3.alignment=Alignment(horizontal='right')


for i in range(1,10):
    ws_invoice[ws_invoice.cell(row=final_invoice.shape[0]+19,column=i).coordinate].border=\
    Border(bottom=Side(style='hair'), left=Side(style='hair'))

for i in range(0,4):
    ws_invoice[ws_invoice.cell(row=final_invoice.shape[0]+19+i,column=10).coordinate].border=\
        Border(bottom=Side(style='hair'), left=Side(style='double'), right=Side(style='thin'))

#Transfer Invoice information
pass_vals(ws_invoice, final_invoice, loc_x=19, loc_y=1)

### Save File

In [77]:
'''
Save file in specified path.
If no path is passed, it will be saved in working directory.
Last portion of path is the document's name - in this case it will be replaced with the invoice number

**.format(invoice_num) - will replace the {} in the path with the invoice number**
'''

#Save file
wb.save('/Users/Xavier/Desktop/Pre_Invoice_{}.xlsx'.format(invoice_num))